For this project, I am using the dataset from UCI Repository SMS Spam Detection: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection. Through the use of PySpark and NLP, I'll be building a spam detection filter of spam message vs ham/good message.

In [1]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName('spam detector').getOrCreate()

# data is separated by tabs
data = spark.read.csv('smsspamcollection/SMSSpamCollection', inferSchema=True, sep='\t')
data = data.withColumnRenamed('_c0', 'class')
data = data.withColumnRenamed('_c1', 'text')
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



#### We will now clean and prepare the data to tokenize.

In [9]:
from pyspark.ml.feature import (Tokenizer, RegexTokenizer, StopWordsRemover, 
VectorAssembler, StringIndexer, IDF, HashingTF, CountVectorizer)

from pyspark.sql.functions import length

In [10]:
data = data.withColumn('tokens', length(data['text']))
data.show()

+-----+--------------------+------+
|class|                text|tokens|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [13]:
# we want to know if there's an actual difference in length b/w a spam text and a ham text.
data.groupby('class').mean().show()

+-----+-----------------+
|class|      avg(tokens)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



#### We can see that there is a clear difference and length is an important feature to consider when doing feature engineering.

In [37]:
# separates the text into a list of words or tokens
tokenizer = Tokenizer(inputCol="text", outputCol="words")

# we will then use StopWordsRemover to remove words from the input that appear frequently but don't carry meaning.
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")

# countVectorizer is used to convert text into a vector of token counts/words count
cv = CountVectorizer(inputCol="filtered_words", outputCol="count_vec")

# idf is used to get the importance of a term to a document in corpus
idf = IDF(inputCol="count_vec", outputCol="tf_idf")

# we will then convert the label of ham/spam into numbers/numeric output
ham_spam_to_num = StringIndexer(inputCol="class", outputCol="label")

#### We will need a vectorAssembler to combine all the numeric features. 

In [43]:
assembler = VectorAssembler(inputCols=["tf_idf", "tokens"], outputCol="features")

### ML Model
##### we will then use the Naive Bayes Classifier.

In [39]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml import Pipeline

In [40]:
nb = NaiveBayes()

In [44]:
# since there's multiple steps, we will use a pipeline
data_prep_pipe = Pipeline(stages=[ham_spam_to_num, tokenizer, remover, cv, idf, assembler])
cleaner = data_prep_pipe.fit(data)

In [45]:
cleaned_data = cleaner.transform(data)

In [50]:
cleaned_data.columns

['class',
 'text',
 'tokens',
 'label',
 'words',
 'filtered_words',
 'count_vec',
 'tf_idf',
 'features']

In [46]:
clean_data = cleaned_data.select(["label", "features"])

In [47]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,120...|
|  1.0|(13424,[8,17,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,96,217...|
|  0.0|(13424,[552,1697,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,47...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [51]:
### we will now train-test split the data
train, test = clean_data.randomSplit([0.7, 0.3])
spam_predictor = nb.fit(train)

In [52]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- tokens: integer (nullable = true)



In [53]:
test_results = spam_predictor.transform(test)
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,5,15,...|[-1006.7150044491...|[1.0,3.6407722850...|       0.0|
|  0.0|(13424,[0,1,7,8,1...|[-1162.4658881613...|[1.0,1.1712555715...|       0.0|
|  0.0|(13424,[0,1,12,33...|[-468.34118912469...|[1.0,1.5188666269...|       0.0|
|  0.0|(13424,[0,1,14,18...|[-1371.5099919464...|[1.0,1.3541023656...|       0.0|
|  0.0|(13424,[0,1,14,31...|[-216.48825201227...|[1.0,1.4301444526...|       0.0|
|  0.0|(13424,[0,1,14,78...|[-688.68692404429...|[1.0,8.2123247162...|       0.0|
|  0.0|(13424,[0,1,15,20...|[-695.14015686622...|[1.0,3.4893348365...|       0.0|
|  0.0|(13424,[0,1,18,20...|[-830.24641873308...|[1.0,1.0384523062...|       0.0|
|  0.0|(13424,[0,1,20,27...|[-985.76769011386...|[0.99999999999973...|       0.0|
|  0.0|(13424,[0

In [60]:
# we will get the accuracy of our prediction using 
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
accuracy_eval = MulticlassClassificationEvaluator()
acc = accuracy_eval.evaluate(test_results)
print("The accuracy of the Naive Bayes Model: {}".format(acc))


The accuracy of the Naive Bayes Model: 0.9191390069960609
